In [27]:
import pandas as pd

In [28]:
# Read the Excel workbook
df = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls')

In [29]:
df.head()

,Food Environment Atlas data download
0,Notes about the Food Environment Atlas downloa...
1,This file contains multiple spreadsheets:
2,1. A variable list that includes metadata abo...
3,2. Spreadsheets that contain data for each of...
4,3. County and State-level supplemental data t...


In [30]:
# Load Access, Stores, Assistance, Insecurity, Local, Health, Restaurants, Socioeconomic data sheets as dataframes 

access = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'ACCESS')
stores = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'STORES')
assistance = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'ASSISTANCE')
insecurity = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'INSECURITY')
local = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'LOCAL')
health = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'HEALTH')
restaurants = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'RESTAURANTS')
socioeconomic = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'SOCIOECONOMIC')
prices_taxes = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'PRICES_TAXES')

In [31]:
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [32]:
restaurants.columns

Index(['FIPS', 'State', 'County', 'FFR09', 'FFR14', 'PCH_FFR_09_14',
       'FFRPTH09', 'FFRPTH14', 'PCH_FFRPTH_09_14', 'FSR09', 'FSR14',
       'PCH_FSR_09_14', 'FSRPTH09', 'FSRPTH14', 'PCH_FSRPTH_09_14',
       'PC_FFRSALES07', 'PC_FFRSALES12', 'PC_FSRSALES07', 'PC_FSRSALES12'],
      dtype='object')

In [33]:
restaurant_cols = ['FIPS', 'State', 'County', 'FFRPTH14', 'FSRPTH14']
stores_cols = ['FIPS', 'State', 'County', 'GROCPTH14', 'SUPERCPTH14', 'CONVSPTH14', 'SPECSPTH14', 'SNAPSPTH16']
access_cols = ['FIPS', 'State', 'County', 'PCT_LACCESS_POP15', 'PCT_LACCESS_LOWI15', 'PCT_LACCESS_HHNV15', 'PCT_LACCESS_SNAP15', 'PCT_LACCESS_CHILD15', 'PCT_LACCESS_SENIORS15', 'PCT_LACCESS_WHITE15', 'PCT_LACCESS_BLACK15', 'PCT_LACCESS_HISP15', 'PCT_LACCESS_NHASIAN15', 'PCT_LACCESS_NHNA15', 'PCT_LACCESS_NHPI15', 'PCT_LACCESS_MULTIR15']
assistance_cols = ['FIPS', 'State', 'County', 'PCT_SNAP16', 'PCT_NSLP15', 'PCT_SBP15', 'PCT_SFSP15', 'PCT_WIC15']
prices_taxes_cols = ['FIPS', 'State', 'County', 'SODATAX_STORES14', 'SODATAX_VENDM14', 'CHIPSTAX_STORES14', 'CHIPSTAX_VENDM14', 'FOOD_TAX14']
local_cols = ['FIPS', 'State', 'County', 'FMRKTPTH16', 'PCT_FMRKT_SNAP16', 'PCT_FMRKT_WIC16', 'PCT_FMRKT_WICCASH16', 'PCT_FMRKT_SFMNP16', 'PCT_FMRKT_CREDIT16', 'PCT_FMRKT_FRVEG16', 'PCT_FMRKT_ANMLPROD16', 'PCT_FMRKT_BAKED16', 'PCT_FMRKT_OTHERFOOD16', 'FOODHUB16']
health_cols = ['FIPS', 'State', 'County', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS13', 'PCT_HSPA15', 'RECFACPTH14']
socioeconomic_cols = ['FIPS', 'State', 'County', 'PCT_NHWHITE10', 'PCT_NHBLACK10', 'PCT_HISP10', 'PCT_NHASIAN10', 'PCT_NHNA10', 'PCT_NHPI10', 'PCT_65OLDER10', 'PCT_18YOUNGER10', 'MEDHHINC15', 'POVRATE15', 'METRO13']


In [34]:
features_new = stores_cols + assistance_cols + local_cols + health_cols + restaurant_cols + socioeconomic_cols + prices_taxes_cols + access_cols
prediction_features = list(set(features_new) - set(['FIPS', 'State', 'County', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS13']))
target = 'PCT_OBESE_ADULTS13'




In [35]:
restaurants.columns

Index(['FIPS', 'State', 'County', 'FFR09', 'FFR14', 'PCH_FFR_09_14',
       'FFRPTH09', 'FFRPTH14', 'PCH_FFRPTH_09_14', 'FSR09', 'FSR14',
       'PCH_FSR_09_14', 'FSRPTH09', 'FSRPTH14', 'PCH_FSRPTH_09_14',
       'PC_FFRSALES07', 'PC_FFRSALES12', 'PC_FSRSALES07', 'PC_FSRSALES12'],
      dtype='object')

In [36]:

dfs = [stores, assistance, insecurity, local, health, restaurants, socioeconomic, prices_taxes]
for df in dfs:
    df.drop(columns=['State', 'County'], axis=1, inplace=True)
    df.set_index('FIPS', inplace=True)



In [37]:
stores.head()

,GROC09,GROC14,PCH_GROC_09_14,GROCPTH09,GROCPTH14,PCH_GROCPTH_09_14,SUPERC09,SUPERC14,PCH_SUPERC_09_14,SUPERCPTH09,...,PCH_SNAPS_12_16,SNAPSPTH12,SNAPSPTH16,PCH_SNAPSPTH_12_16,WICS08,WICS12,PCH_WICS_08_12,WICSPTH08,WICSPTH12,PCH_WICSPTH_08_12
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,6,4,-33.333333,0.110834,0.072209,-34.849716,1,1,0.0,0.018472,...,12.694878,0.674004,0.760911,12.894172,6,5,-16.66667,0.119156,0.090067,-24.412460
1003,24,29,20.833333,0.133775,0.144920,8.331001,6,6,0.0,0.033444,...,43.192771,0.725055,0.949753,30.990390,25,27,8.00000,0.141875,0.141517,-0.252126
1005,5,5,0.000000,0.180786,0.185963,2.863838,0,1,NaN,0.000000,...,0.956938,1.280590,1.354387,5.762745,6,7,16.66667,0.201099,0.257344,27.968330
1007,6,5,-16.666667,0.261540,0.222163,-15.055985,1,1,0.0,0.043590,...,20.512821,0.719122,0.864874,20.267995,6,5,-16.66667,0.277919,0.221268,-20.383970
1009,6,6,0.000000,0.104637,0.103952,-0.654897,1,1,0.0,0.017440,...,23.903509,0.657144,0.815946,24.165470,10,6,-40.00000,0.173028,0.103760,-40.033200


In [38]:
access.set_index('FIPS', drop=False, inplace=True)

In [39]:
access.head()

,FIPS,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,...,LACCESS_HISP15,PCT_LACCESS_HISP15,LACCESS_NHASIAN15,PCT_LACCESS_NHASIAN15,LACCESS_NHNA15,PCT_LACCESS_NHNA15,LACCESS_NHPI15,PCT_LACCESS_NHPI15,LACCESS_MULTIR15,PCT_LACCESS_MULTIR15
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,...,471.136164,0.863345,86.767975,0.159000,61.169869,0.112092,8.817961,0.016159,482.848633,0.884808
1003,1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,...,1377.874834,0.755973,212.946378,0.116833,181.649648,0.099662,14.819634,0.008131,1127.696098,0.618712
1005,1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,...,509.377525,1.855183,17.096410,0.062266,39.960527,0.145539,8.082376,0.029436,462.382655,1.684025
1007,1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,...,8.596762,0.037516,1.994318,0.008703,2.513097,0.010967,0.000000,0.000000,5.259244,0.022951
1009,1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,...,497.489891,0.867886,8.428994,0.014705,28.938242,0.050484,1.062851,0.001854,202.914187,0.353990


In [15]:
stores.head()

,GROC09,GROC14,PCH_GROC_09_14,GROCPTH09,GROCPTH14,PCH_GROCPTH_09_14,SUPERC09,SUPERC14,PCH_SUPERC_09_14,SUPERCPTH09,...,PCH_SNAPS_12_16,SNAPSPTH12,SNAPSPTH16,PCH_SNAPSPTH_12_16,WICS08,WICS12,PCH_WICS_08_12,WICSPTH08,WICSPTH12,PCH_WICSPTH_08_12
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,6,4,-33.333333,0.110834,0.072209,-34.849716,1,1,0.0,0.018472,...,12.694878,0.674004,0.760911,12.894172,6,5,-16.66667,0.119156,0.090067,-24.412460
1003,24,29,20.833333,0.133775,0.144920,8.331001,6,6,0.0,0.033444,...,43.192771,0.725055,0.949753,30.990390,25,27,8.00000,0.141875,0.141517,-0.252126
1005,5,5,0.000000,0.180786,0.185963,2.863838,0,1,NaN,0.000000,...,0.956938,1.280590,1.354387,5.762745,6,7,16.66667,0.201099,0.257344,27.968330
1007,6,5,-16.666667,0.261540,0.222163,-15.055985,1,1,0.0,0.043590,...,20.512821,0.719122,0.864874,20.267995,6,5,-16.66667,0.277919,0.221268,-20.383970
1009,6,6,0.000000,0.104637,0.103952,-0.654897,1,1,0.0,0.017440,...,23.903509,0.657144,0.815946,24.165470,10,6,-40.00000,0.173028,0.103760,-40.033200


In [40]:
# Combine all sheets into one dataframe by joining on FIPS col.

#master_df = []
#for df in dfs:
#    master_df = access.join(df, on="FIPS")
# In [37]: df1.merge(df2, right_index=True, left_on='v2', how='outer')

master_df = access.join(dfs)

In [41]:
master_df.count()

FIPS                         3143
State                        3143
County                       3143
LACCESS_POP10                3143
LACCESS_POP15                3124
PCH_LACCESS_POP_10_15        3117
PCT_LACCESS_POP10            3143
PCT_LACCESS_POP15            3124
LACCESS_LOWI10               3143
LACCESS_LOWI15               3123
PCH_LACCESS_LOWI_10_15       3115
PCT_LACCESS_LOWI10           3143
PCT_LACCESS_LOWI15           3123
LACCESS_HHNV10               3143
LACCESS_HHNV15               3140
PCH_LACCESS_HHNV_10_15       3129
PCT_LACCESS_HHNV10           3143
PCT_LACCESS_HHNV15           3140
LACCESS_SNAP15               3123
PCT_LACCESS_SNAP15           3123
LACCESS_CHILD10              3143
LACCESS_CHILD15              3124
LACCESS_CHILD_10_15          3115
PCT_LACCESS_CHILD10          3143
PCT_LACCESS_CHILD15          3124
LACCESS_SENIORS10            3143
LACCESS_SENIORS15            3124
PCH_LACCESS_SENIORS_10_15    3117
PCT_LACCESS_SENIORS10        3143
PCT_LACCESS_SE

In [42]:
access.head()

,FIPS,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,...,LACCESS_HISP15,PCT_LACCESS_HISP15,LACCESS_NHASIAN15,PCT_LACCESS_NHASIAN15,LACCESS_NHNA15,PCT_LACCESS_NHNA15,LACCESS_NHPI15,PCT_LACCESS_NHPI15,LACCESS_MULTIR15,PCT_LACCESS_MULTIR15
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,...,471.136164,0.863345,86.767975,0.159000,61.169869,0.112092,8.817961,0.016159,482.848633,0.884808
1003,1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,...,1377.874834,0.755973,212.946378,0.116833,181.649648,0.099662,14.819634,0.008131,1127.696098,0.618712
1005,1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,...,509.377525,1.855183,17.096410,0.062266,39.960527,0.145539,8.082376,0.029436,462.382655,1.684025
1007,1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,...,8.596762,0.037516,1.994318,0.008703,2.513097,0.010967,0.000000,0.000000,5.259244,0.022951
1009,1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,...,497.489891,0.867886,8.428994,0.014705,28.938242,0.050484,1.062851,0.001854,202.914187,0.353990


In [43]:
master_df['County'].dtype

dtype('O')

In [20]:
master_df.head()

,FIPS,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,...,METRO13,POPLOSS10,MILK_PRICE10,SODA_PRICE10,MILK_SODA_PRICE10,SODATAX_STORES14,SODATAX_VENDM14,CHIPSTAX_STORES14,CHIPSTAX_VENDM14,FOOD_TAX14
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,...,1,0.0,0.970305,0.972216,0.923289,4.0,4.0,4.0,4.0,4.0
1003,1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,...,1,0.0,1.017690,1.001391,0.940165,4.0,4.0,4.0,4.0,4.0
1005,1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,...,0,0.0,1.136671,0.992584,1.059400,4.0,4.0,4.0,4.0,4.0
1007,1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,...,1,0.0,0.970305,0.972216,0.923289,4.0,4.0,4.0,4.0,4.0
1009,1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,...,1,0.0,0.970305,0.972216,0.923289,4.0,4.0,4.0,4.0,4.0


In [44]:
master_df.shape

(3143, 280)

In [45]:
df = master_df[list(set(features_new))]
df.shape

(3143, 59)

In [ ]:
df.tail()

In [23]:
df.isnull().sum()

PCT_DIABETES_ADULTS13       1
PCT_HSPA15               1118
PCT_LACCESS_HHNV15          3
PCT_NSLP15                  0
PCT_FMRKT_FRVEG16         895
PCT_OBESE_ADULTS13          1
PCT_LACCESS_WHITE15        19
PCT_NHNA10                  0
PCT_FMRKT_SFMNP16         895
SPECSPTH14                  0
PCT_LACCESS_CHILD15        19
PCT_LACCESS_MULTIR15       19
SODATAX_VENDM14             0
PCT_NHBLACK10               0
PCT_SFSP15                  0
SNAPSPTH16                 29
PCT_LACCESS_SENIORS15      19
PCT_FMRKT_WICCASH16       895
PCT_LACCESS_BLACK15        19
State                       0
SUPERCPTH14                 0
PCT_LACCESS_POP15          19
FOODHUB16                   0
PCT_NHWHITE10               0
PCT_FMRKT_BAKED16         895
FFRPTH14                    0
PCT_NHPI10                  0
POVRATE15                   4
PCT_LACCESS_SNAP15         20
PCT_LACCESS_HISP15         19
FMRKTPTH16                  2
FOOD_TAX14                  0
METRO13                     0
SODATAX_ST

In [46]:
# Food Tax Rate, Fast Food Restaurants, Farmer's Markers
cols_required_for_prediction = ['FOOD_TAX14', 'FFRPTH14', 'FMRKTPTH16']
df = df.dropna(subset=cols_required_for_prediction)
df.isnull().sum()

PCT_DIABETES_ADULTS13       0
PCT_HSPA15               1118
PCT_LACCESS_HHNV15          2
PCT_NSLP15                  0
PCT_FMRKT_FRVEG16         895
PCT_OBESE_ADULTS13          0
PCT_LACCESS_WHITE15        19
PCT_NHNA10                  0
PCT_FMRKT_SFMNP16         895
SPECSPTH14                  0
PCT_LACCESS_CHILD15        19
PCT_LACCESS_MULTIR15       19
SODATAX_VENDM14             0
PCT_NHBLACK10               0
PCT_SFSP15                  0
SNAPSPTH16                 27
PCT_LACCESS_SENIORS15      19
PCT_FMRKT_WICCASH16       895
PCT_LACCESS_BLACK15        19
State                       0
SUPERCPTH14                 0
PCT_LACCESS_POP15          19
FOODHUB16                   0
PCT_NHWHITE10               0
PCT_FMRKT_BAKED16         895
FFRPTH14                    0
PCT_NHPI10                  0
POVRATE15                   2
PCT_LACCESS_SNAP15         19
PCT_LACCESS_HISP15         19
FMRKTPTH16                  0
FOOD_TAX14                  0
METRO13                     0
SODATAX_ST

In [47]:
df.count()

PCT_DIABETES_ADULTS13    3141
PCT_HSPA15               2023
PCT_LACCESS_HHNV15       3139
PCT_NSLP15               3141
PCT_FMRKT_FRVEG16        2246
PCT_OBESE_ADULTS13       3141
PCT_LACCESS_WHITE15      3122
PCT_NHNA10               3141
PCT_FMRKT_SFMNP16        2246
SPECSPTH14               3141
PCT_LACCESS_CHILD15      3122
PCT_LACCESS_MULTIR15     3122
SODATAX_VENDM14          3141
PCT_NHBLACK10            3141
PCT_SFSP15               3141
SNAPSPTH16               3114
PCT_LACCESS_SENIORS15    3122
PCT_FMRKT_WICCASH16      2246
PCT_LACCESS_BLACK15      3122
State                    3141
SUPERCPTH14              3141
PCT_LACCESS_POP15        3122
FOODHUB16                3141
PCT_NHWHITE10            3141
PCT_FMRKT_BAKED16        2246
FFRPTH14                 3141
PCT_NHPI10               3141
POVRATE15                3139
PCT_LACCESS_SNAP15       3122
PCT_LACCESS_HISP15       3122
FMRKTPTH16               3141
FOOD_TAX14               3141
METRO13                  3141
SODATAX_ST

In [48]:
# make sure metro is boolean
df['METRO13'] = df['METRO13'].astype('bool')

In [49]:
print(df.dtypes)

PCT_DIABETES_ADULTS13    float64
PCT_HSPA15               float64
PCT_LACCESS_HHNV15       float64
PCT_NSLP15               float64
PCT_FMRKT_FRVEG16        float64
PCT_OBESE_ADULTS13       float64
PCT_LACCESS_WHITE15      float64
PCT_NHNA10               float64
PCT_FMRKT_SFMNP16        float64
SPECSPTH14               float64
PCT_LACCESS_CHILD15      float64
PCT_LACCESS_MULTIR15     float64
SODATAX_VENDM14          float64
PCT_NHBLACK10            float64
PCT_SFSP15               float64
SNAPSPTH16               float64
PCT_LACCESS_SENIORS15    float64
PCT_FMRKT_WICCASH16      float64
PCT_LACCESS_BLACK15      float64
State                     object
SUPERCPTH14              float64
PCT_LACCESS_POP15        float64
FOODHUB16                  int64
PCT_NHWHITE10            float64
PCT_FMRKT_BAKED16        float64
FFRPTH14                 float64
PCT_NHPI10               float64
POVRATE15                float64
PCT_LACCESS_SNAP15       float64
PCT_LACCESS_HISP15       float64
FMRKTPTH16

In [50]:
df.to_csv('data/df_for_webapp.csv', index=False)

In [51]:
df_test = df

In [52]:
df_test['County'] = (df_test['County'].values)

In [53]:
df_test['County'].values

array(['Autauga', 'Baldwin', 'Barbour', ..., 'Uinta', 'Washakie',
       'Weston'], dtype=object)

In [54]:
df_test.head()

,PCT_DIABETES_ADULTS13,PCT_HSPA15,PCT_LACCESS_HHNV15,PCT_NSLP15,PCT_FMRKT_FRVEG16,PCT_OBESE_ADULTS13,PCT_LACCESS_WHITE15,PCT_NHNA10,PCT_FMRKT_SFMNP16,SPECSPTH14,...,PCT_LACCESS_LOWI15,FSRPTH14,PCT_LACCESS_NHNA15,PCT_18YOUNGER10,PCT_LACCESS_NHPI15,CHIPSTAX_VENDM14,GROCPTH14,PCT_FMRKT_ANMLPROD16,FIPS,PCT_FMRKT_SNAP16
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,13.0,25.4,3.351332,10.934529,100.0,34.1,23.163613,0.397647,0.0,0.036104,...,11.991125,0.523513,0.112092,26.777959,0.016159,4.0,0.072209,0.0,1001,0.0
1003,10.4,25.4,1.905114,10.934529,75.0,27.4,13.981393,0.628755,100.0,0.129928,...,5.424427,1.104387,0.099662,22.987408,0.008131,4.0,0.144920,100.0,1003,0.0
1005,18.4,25.4,4.329378,10.934529,50.0,44.4,10.302934,0.218524,50.0,0.074385,...,10.739667,0.557890,0.145539,21.906982,0.029436,4.0,0.185963,0.0,1005,50.0
1007,14.8,25.4,2.821427,10.934529,0.0,40.3,2.358770,0.279293,0.0,0.044433,...,2.601627,0.222163,0.010967,22.696923,0.000000,4.0,0.222163,0.0,1007,0.0
1009,14.1,25.4,3.336414,10.934529,100.0,34.6,5.909147,0.497191,100.0,0.000000,...,2.880150,0.259880,0.050484,24.608353,0.001854,4.0,0.103952,100.0,1009,0.0


In [ ]:
print ((df[df['County'] == 'Orange'].County.values))

In [ ]:
for county in df['County'].values:
    str(county)

df['County'].values

In [ ]:
df['State'] = df['State'].astype('category')

In [ ]:
# df.to_csv('data/df_for_webapp.csv', index=False)